In [22]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

import captcha_params

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.optimizers import adamax
from keras.callbacks import Callback
from keras import backend as K
from load_data import *
import h5py
from keras.models import model_from_json

In [14]:
batch_size = 10
nb_epoch = 100

MAX_CAPTCHA = captcha_params.get_captcha_size()
CHAR_SET_LEN = captcha_params.get_char_set_len()

# input image dimensions
img_rows, img_cols = captcha_params.get_height(), captcha_params.get_width()

In [16]:
from load_data import load_data
(X_train, y_train), (X_test, y_test) = load_data(tol_num = 1000,train_num = 700,folder='labeled_captchas')

In [17]:
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols,1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [18]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train =(X_train/255)-0.5
X_test =(X_test/255)-0.5
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (700, 150, 330, 1)
700 train samples
300 test samples


In [28]:
model = Sequential()

In [29]:
model.add(Conv2D(kernel_size=kernel_size, filters=nb_filters1, padding='same' ,input_shape=input_shape ))
model.add(LeakyReLU(0.1))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Conv2D(16, kernel_size=kernel_size))
model.add(LeakyReLU(0.1))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Conv2D(8, kernel_size=kernel_size))
model.add(LeakyReLU(0.1))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Conv2D(4, kernel_size=kernel_size))
model.add(LeakyReLU(0.1))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

# Fully connected layer
model.add(Flatten())
model.add(Dense(512))
model.add(LeakyReLU(0.1))
model.add(Dropout(0.25))
model.add(Dense(MAX_CAPTCHA*CHAR_SET_LEN))
model.add(Activation('softmax'))

In [32]:
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)
batch_size =70

INIT_LR = 5e-3
model.compile(loss='categorical_crossentropy',
              optimizer=adamax(lr=INIT_LR),
              metrics=['accuracy'])

# scheduler of learning rate (decay with epochs)
def lr_scheduler(epoch):
    return INIT_LR * 0.9 ** epoch

# callback for printing of actual learning rate used by optimizer
class LrHistory(Callback):
    def on_epoch_begin(self, epoch, logs={}):
        print("Learning rate:", K.get_value(model.optimizer.lr))

In [33]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epoch,
            verbose=1, validation_data=(X_test,y_test))

Train on 700 samples, validate on 300 samples
Epoch 1/100
700/700 [==============================] - 128s 182ms/step - loss: 137.2150 - acc: 0.0071 - val_loss: 157.2420 - val_acc: 0.0000e+00
Epoch 2/100
700/700 [==============================] - 117s 167ms/step - loss: 126.7715 - acc: 0.0100 - val_loss: 157.0610 - val_acc: 0.0000e+00
Epoch 3/100
700/700 [==============================] - 117s 168ms/step - loss: 123.6649 - acc: 0.0000e+00 - val_loss: 160.8435 - val_acc: 0.0000e+00
Epoch 4/100
700/700 [==============================] - 118s 168ms/step - loss: 118.9386 - acc: 0.0000e+00 - val_loss: 163.6640 - val_acc: 0.0000e+00
Epoch 5/100
700/700 [==============================] - 117s 167ms/step - loss: 115.7304 - acc: 0.0014 - val_loss: 167.0008 - val_acc: 0.0000e+00
Epoch 6/100
700/700 [==============================] - 126s 180ms/step - loss: 114.3099 - acc: 0.0000e+00 - val_loss: 171.8665 - val_acc: 0.0000e+00
Epoch 7/100
700/700 [==============================] - 157s 224ms/step -

KeyboardInterrupt: 